In [1]:
import sys
import os
import numpy as np
from sklearn import metrics
sys.path.append("..")
from SINC_functions import *

In [7]:
# import warnings
# warnings.filterwarnings("ignore",category = DeprecationWarning, module = user_ns.get("__name__"))

def TPR_FPR_B(phi,B):
    B_est = np.absolute(phi) > 0.050
    B_true = B != 0

    TP = np.sum(((phi > 0.50) == 1) & ((B != 0) == 1))  * 1.0
    FP = np.sum(((phi > 0.50) == 1) & ((B != 0) == 0))  * 1.0
    FN = np.sum(((phi > 0.50) == 0) & ((B != 0) == 1))  * 1.0
    TN = np.sum(((phi > 0.50) == 0) & ((B != 0) == 0))  * 1.0
    NN = np.sum(B_true == 0) * 1.0
    NP = np.sum(B_true == 1) * 1.0

    prec = TP / (TP + FP)
    recall = TP / (TP + FN)

    F1 = 2*(prec*recall) / (prec + recall)

    MCC = ((TP*TN) - (FP*FN))/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))

    TPR = TP/NP
    FPR = FP/(FP + NN)
    return(TPR,FPR,F1,MCC)

## get shapes for simulation
B_true = np.genfromtxt("./SimulatedData/Cluster/B1.csv")
X_true = np.genfromtxt("./SimulatedData/Cluster/x1.csv")[:,0:30]

N = X_true.shape[0]
P = X_true.shape[1]
Q = B_true.shape[1]


reps = 10
goal = .10
adj_est_all = np.zeros((P, P, reps))
EZ_all = np.zeros((P, P, reps))
B_est_all = np.zeros((P,Q,reps))
phi_all = np.copy(B_est_all)
spars = np.zeros(reps)
v0s = np.linspace(.0001, .01, reps)
TPR = np.zeros(reps)
FPR = np.zeros(reps)
from itertools import chain

results = np.zeros((25, 5))
results_B = np.zeros((25,4))
total_iters = np.zeros(reps)

Z_track_all = []
omega_track_all = []
B_track_all = []

for sim in range(1):
    TPR = np.zeros(reps)
    FPR = np.zeros(reps)
    iters = np.zeros(reps)
    spars = np.zeros(reps)
    
    v1 = 10
    lamb = 150
    vB = 1
    max_iters = 50
    tol_prec = 0.01
    tol_B = 0.01
    cpus = 1

    print("**********************start iteration " + str(sim) + "*************************")
    x = np.genfromtxt("./SimulatedData/Cluster/x" + str(2 + sim) + ".csv")
    m = np.genfromtxt("./SimulatedData/Cluster/m" + str(2 + sim) + ".csv")
    adj_true = np.genfromtxt("./SimulatedData/Cluster/adj" + str(2 + sim) + ".csv")
    adj_true = adj_true[0:30,0:30]
    B_true = np.genfromtxt("./SimulatedData/Cluster/B" + str(2 + sim) + ".csv")

    for v in range(reps):
        
        v0 = v0s[v]

        omega, EZ, phi,B,iters_total  = SINC(x[:,0:30], m, v0, v1, lamb, vB, max_iters, tol_prec, tol_B, cpus)
        
        B_est_all[:,:,v] = B
        phi_all[:,:,v] = phi
        EZ_all[:, :, v] = EZ
        adj_est = (EZ > 0.50)*1.0
        spars[v] = np.sum(adj_est) / 2
        TPR[v], FPR[v] = TPR_FPR(adj_true, adj_est)
        if np.sum(adj_est) / 2 <= 5:
            break

    adj_est_all = (EZ_all > 0.50) * 1
    ind = np.argmin(np.absolute((np.sum(adj_est_all, (0,1)) / (12 * 11 / 2.0) / 2) - .10))
    EZ = EZ_all[:, :, ind]
    phi_ind = phi_all[:,:,ind]
    total_iters[v] = iters[ind]


    TPR = np.append(1, np.append(TPR, 0))
    FPR = np.append(1, np.append(FPR, 0))
    auc = metrics.auc(np.array(FPR),np.array(TPR))

    auc_v0 = 0
    n = len(TPR)
    for n in range(n - 1):
        b1 = TPR[n]
        b2 = TPR[n + 1]
        h = FPR[n] - FPR[n + 1]
        auc_v0 += (b1 + b2) * h / 2

    adj_est = adj_est_all[:, :, ind]
    TPR, FPR = TPR_FPR(adj_true, adj_est)
    f1, mcc = F1(adj_true, adj_est)
    results[sim,] = [TPR, FPR, f1, mcc, auc]
    B_edges_true = (B_true[0:30,:] != 0)*1.0
    B_edges_est = (phi_ind > 0.50) *1
    results_B[sim,] = Performance_B(B_edges_true,B_edges_est)

    np.savetxt("./Results/NetworkResults_Cluster.csv",results, delimiter=",")
    np.savetxt("./Results/CoefficientResults_Cluster.csv",results_B, delimiter=",")


**********************start iteration 0*************************
('Finished Iteration 1: ', 'change in Omega,B,Z', 0.5099765864068562, 0.7643514374532351, 3.7056732061660957)
('Finished Iteration 2: ', 'change in Omega,B,Z', 0.5186342990870856, 0.19744108336017607, 1.2626743332653407)
('Finished Iteration 3: ', 'change in Omega,B,Z', 0.15414567115854083, 0.10931617043245467, 1.0701551048975677)
('Finished Iteration 4: ', 'change in Omega,B,Z', 0.12855070496348475, 0.14666229984737295, 1.232814762287267)
('Finished Iteration 5: ', 'change in Omega,B,Z', 0.09953634116473409, 0.1490724529150475, 0.7464388170723337)
('Finished Iteration 6: ', 'change in Omega,B,Z', 0.05371358983059893, 0.09082718051811417, 0.513649310616648)
('Finished Iteration 7: ', 'change in Omega,B,Z', 0.029872588724143778, 0.061166694200436855, 0.5774445072389569)
('Finished Iteration 8: ', 'change in Omega,B,Z', 0.016312059627769937, 0.11930370108586283, 0.37744478627995504)
('Finished Iteration 9: ', 'change in Ome

('Finished Iteration 19: ', 'change in Omega,B,Z', 0.046134570716032997, 0.12335962095798231, 0.4106531597666714)
('Finished Iteration 20: ', 'change in Omega,B,Z', 0.029220057333928295, 0.08058917691805956, 0.2930106546887554)
('Finished Iteration 21: ', 'change in Omega,B,Z', 0.016952355854938705, 0.05453091954694467, 0.21258601606077798)
('Finished Iteration 22: ', 'change in Omega,B,Z', 0.010029528111341612, 0.03786304376261329, 0.15760865128696722)
('Finished Iteration 23: ', 'change in Omega,B,Z', 0.006251704854256079, 0.027047630314018967, 0.11985118586535526)
('Finished Iteration 24: ', 'change in Omega,B,Z', 0.0039867874510035595, 0.019634362033156272, 0.09244625085789338)
('Finished Iteration 25: ', 'change in Omega,B,Z', 0.0025348933478557267, 0.01456019261973851, 0.07318222907192728)
('Finished Iteration 26: ', 'change in Omega,B,Z', 0.0016197717667378875, 0.010969083007173919, 0.06051778038417743)
('Finished Iteration 27: ', 'change in Omega,B,Z', 0.0015967208694787072, 0.

('Finished Iteration 1: ', 'change in Omega,B,Z', 0.5120274469170076, 0.7643514374532351, 3.7037796855340828)
('Finished Iteration 2: ', 'change in Omega,B,Z', 0.5208299964596963, 0.19487296325826392, 1.2758004151824864)
('Finished Iteration 3: ', 'change in Omega,B,Z', 0.16289557313423608, 0.11660949787961228, 0.6371883539434569)
('Finished Iteration 4: ', 'change in Omega,B,Z', 0.12287339384294005, 0.13412677504445086, 0.847698567010837)
('Finished Iteration 5: ', 'change in Omega,B,Z', 0.07689110108324071, 0.14797274841548236, 0.7224399148478984)
('Finished Iteration 6: ', 'change in Omega,B,Z', 0.0469385438479768, 0.1480922396059233, 0.44941545053874554)
('Finished Iteration 7: ', 'change in Omega,B,Z', 0.030526764572468945, 0.1001204327362829, 0.3311745519824001)
('Finished Iteration 8: ', 'change in Omega,B,Z', 0.01766308379890491, 0.0704837671490473, 0.2559948128981264)
('Finished Iteration 9: ', 'change in Omega,B,Z', 0.06280065685915237, 0.05156812836187197, 0.8968267248722307

('Finished Iteration 8: ', 'change in Omega,B,Z', 0.018265220626340284, 0.07294098302827046, 0.26279035455906197)
('Finished Iteration 9: ', 'change in Omega,B,Z', 0.06181016065550482, 0.05338577851836024, 0.8419338451593816)
('Finished Iteration 10: ', 'change in Omega,B,Z', 0.08419439825777641, 0.17509328635217714, 0.736501218138825)
('Finished Iteration 11: ', 'change in Omega,B,Z', 0.04881132440265734, 0.1381262503026952, 0.4364286789628442)
('Finished Iteration 12: ', 'change in Omega,B,Z', 0.03147788060317901, 0.07162015076499234, 0.2567702731151753)
('Finished Iteration 13: ', 'change in Omega,B,Z', 0.008398622144082157, 0.03991154397437785, 0.16163045319695346)
('Finished Iteration 14: ', 'change in Omega,B,Z', 0.011975404780888946, 0.026397782996084085, 0.11096933020243238)
('Finished Iteration 15: ', 'change in Omega,B,Z', 0.029265406967954773, 0.02396503807610184, 0.5556175097806334)
('Finished Iteration 16: ', 'change in Omega,B,Z', 0.05950691100098704, 0.16415879498696107,

('Finished Iteration 22: ', 'change in Omega,B,Z', 0.0031783236320030994, 0.03493205976782138, 0.10902953989851816)
('Finished Iteration 23: ', 'change in Omega,B,Z', 0.0030423230248080024, 0.023487507286844678, 0.073338502249221)
('Finished Iteration 24: ', 'change in Omega,B,Z', 0.0028866814244690175, 0.01611425422202717, 0.05025228168463469)
('Finished Iteration 25: ', 'change in Omega,B,Z', 0.0027093058322780195, 0.011176830815941674, 0.03659595803173277)
('Finished Iteration 26: ', 'change in Omega,B,Z', 0.03663476098266272, 0.009223657366198346, 0.6602481376404041)
('Finished Iteration 27: ', 'change in Omega,B,Z', 0.03478447934728002, 0.14712537309965013, 0.389432420271683)
('Finished Iteration 28: ', 'change in Omega,B,Z', 0.021746907922503445, 0.09117188440504959, 0.26325007359291375)
('Finished Iteration 29: ', 'change in Omega,B,Z', 0.01212870762867524, 0.05821070557106217, 0.1643717049116291)
('Finished Iteration 30: ', 'change in Omega,B,Z', 0.006261201570525232, 0.0379017

In [5]:
adj_est

array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [6]:
adj_true

array([[0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0.]])